In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions.categorical as C
import torch.optim as optim




In [ ]:
class Model(nn.Module):
  def __init__(self):
     super(Model,self).__init__()
     self.fc1= nn.Linear(1,3)
     self.fc2 =nn.Linear(3,3)
  def forward(self,x):
     out= self.fc1(x)
     #out = F.relu(out)
     out = self.fc2(out)
     out = F.softmax(out)
     return out








In [ ]:
input = torch.ones(1)
model = Model()
opt = optim.Adam(model.parameters(),lr=1e-3)



In [ ]:
def training_loop(p):
  for i in range(3000):
      entry = np.random.choice([0.0,1.0,2.0],p=p)
      entry = torch.tensor(entry)
      #print(entry)
      out = model(input)

      if(entry==0.0):
        reward = out[1]*20.0 +out[2]*5.0 + out[0]*10
      elif(entry==1.0):
        reward = out[0]*5.0 + out[2]*20.0 +out[1]*10.0
      elif(entry==2.0):
        reward = out[0]*20.0 + out[1]*5.0 + out[2]*10.0

      loss = -torch.log(reward)
      opt.zero_grad()
      loss.backward()
      opt.step()


  print(out)
  return out






In [ ]:
def generate_p():
  p1 = np.random.uniform(0,1.0)
  p2 =np.random.uniform(0,1-p1)
  p3 = 1-(p1+p2)
  p =[p1,p2,p3]
  p = np.random.permutation(p)
  return p

def compute_reward(out,p):
    reward = 0

    for i in range(3000):
      entry = np.random.choice([0.0,1.0,2.0],p=p)
      x = torch.multinomial(out,num_samples=1)

      if(x==0):

        if(entry==2):
          reward+=1

        elif(entry==1):
          reward-=1
      elif(x==1):

        if(entry==0):
          reward+=1
        elif(entry==2):
          reward-=1

      elif(x==2):

        if(entry==0):
          reward-=1
        elif(entry==1):
          reward+=1
    return reward





In [ ]:
data = []
for i in range(1500):
  p = generate_p()
  input = torch.ones(1)
  model = Model()
  opt = optim.Adam(model.parameters(),lr=1e-3)
  out = training_loop(p)
  reward = compute_reward(out,p)
  data.append((p,out,reward))









<ipython-input-2-7e42ea2fb918>:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


tensor([2.0339e-03, 8.5272e-04, 9.9711e-01], grad_fn=<SoftmaxBackward0>)
tensor([9.8897e-03, 9.8966e-01, 4.5445e-04], grad_fn=<SoftmaxBackward0>)
tensor([3.9253e-04, 9.8550e-02, 9.0106e-01], grad_fn=<SoftmaxBackward0>)
tensor([1.7496e-03, 9.9740e-01, 8.4828e-04], grad_fn=<SoftmaxBackward0>)
tensor([6.1943e-04, 9.9892e-01, 4.5974e-04], grad_fn=<SoftmaxBackward0>)
tensor([0.3150, 0.1899, 0.4951], grad_fn=<SoftmaxBackward0>)
tensor([6.2316e-04, 5.4034e-04, 9.9884e-01], grad_fn=<SoftmaxBackward0>)
tensor([2.7090e-04, 9.9922e-01, 5.0829e-04], grad_fn=<SoftmaxBackward0>)
tensor([1.3210e-03, 8.7131e-04, 9.9781e-01], grad_fn=<SoftmaxBackward0>)
tensor([3.7884e-04, 4.0741e-04, 9.9921e-01], grad_fn=<SoftmaxBackward0>)
tensor([0.0024, 0.8234, 0.1742], grad_fn=<SoftmaxBackward0>)
tensor([0.0010, 0.9962, 0.0028], grad_fn=<SoftmaxBackward0>)
tensor([0.9796, 0.0192, 0.0012], grad_fn=<SoftmaxBackward0>)
tensor([0.0018, 0.9941, 0.0041], grad_fn=<SoftmaxBackward0>)
tensor([0.0103, 0.0065, 0.9833], grad_

KeyboardInterrupt: ignored

In [ ]:
print(len(data))

106


In [ ]:
class exploit(nn.Module):
   def __init__(self):
     super(exploit,self).__init__()
     self.fc1= nn.Linear(3,30)
     self.fc2 = nn.Linear(30,30)
     self.fc3 =nn.Linear(30,4)

   def forward(self,x):
      out= self.fc1(x)
      out = F.relu(out)
      out = self.fc2(out)
      out = F.relu(out)
      out = self.fc3(out)
      prob = F.softmax(out[:,0:3])
      r = out[:,-1]
      return prob,r



In [ ]:
data_probs = []
data_out = []
data_rewards = []
model_pred = exploit()
opt = optim.Adam(model_pred.parameters(),lr=1e-4)
for entry in data:
   data_probs.append(entry[0])
   data_out.append(entry[1].detach().numpy())
   data_rewards.append(entry[2])

data_probs = np.array(data_probs)
data_out = np.array(data_out)
train_data = data_probs
#valid_data = data_probs[]
train_target = data_out
#valid_target = data_out[1000:]
take = np.arange(len(train_data))
batches = len(train_data)//64
criterion = torch.nn.CrossEntropyLoss()
for epochs in range(1000):
    take = np.random.permutation(take)
    j=0
    in_ = train_data[take]
    out = train_target[take]
    for i in range(batches):
      print("Enter")
      x = in_[j:j+64]
      x = torch.tensor(x,dtype=torch.float)
      #print(x.shape)
      pred,_ = model_pred(x)
      #print(pred.shape)
      target = out[j:j+64]
      target=torch.tensor(target)
      loss = criterion(pred,target)
      opt.zero_grad()
      loss.backward()
      opt.step()
      j=j+64
    print(loss)


Enter
tensor(1.1020, grad_fn=<DivBackward1>)
Enter
tensor(1.1026, grad_fn=<DivBackward1>)
Enter
tensor(1.1031, grad_fn=<DivBackward1>)
Enter
tensor(1.1026, grad_fn=<DivBackward1>)
Enter
tensor(1.1012, grad_fn=<DivBackward1>)
Enter
tensor(1.1026, grad_fn=<DivBackward1>)
Enter
tensor(1.1030, grad_fn=<DivBackward1>)
Enter
tensor(1.1021, grad_fn=<DivBackward1>)
Enter
tensor(1.1018, grad_fn=<DivBackward1>)
Enter
tensor(1.1025, grad_fn=<DivBackward1>)
Enter
tensor(1.1025, grad_fn=<DivBackward1>)
Enter
tensor(1.1007, grad_fn=<DivBackward1>)
Enter
tensor(1.1023, grad_fn=<DivBackward1>)
Enter
tensor(1.1019, grad_fn=<DivBackward1>)
Enter
tensor(1.1023, grad_fn=<DivBackward1>)
Enter
tensor(1.1011, grad_fn=<DivBackward1>)
Enter
tensor(1.1007, grad_fn=<DivBackward1>)
Enter
tensor(1.1015, grad_fn=<DivBackward1>)
Enter
tensor(1.1015, grad_fn=<DivBackward1>)
Enter
tensor(1.1012, grad_fn=<DivBackward1>)
Enter
tensor(1.1021, grad_fn=<DivBackward1>)
Enter
tensor(1.1022, grad_fn=<DivBackward1>)
Enter
tens

<ipython-input-8-9e42e92374fe>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(out[:,0:3])


Enter
tensor(1.0953, grad_fn=<DivBackward1>)
Enter
tensor(1.0976, grad_fn=<DivBackward1>)
Enter
tensor(1.0948, grad_fn=<DivBackward1>)
Enter
tensor(1.0960, grad_fn=<DivBackward1>)
Enter
tensor(1.0955, grad_fn=<DivBackward1>)
Enter
tensor(1.0951, grad_fn=<DivBackward1>)
Enter
tensor(1.0951, grad_fn=<DivBackward1>)
Enter
tensor(1.0944, grad_fn=<DivBackward1>)
Enter
tensor(1.0956, grad_fn=<DivBackward1>)
Enter
tensor(1.0956, grad_fn=<DivBackward1>)
Enter
tensor(1.0957, grad_fn=<DivBackward1>)
Enter
tensor(1.0953, grad_fn=<DivBackward1>)
Enter
tensor(1.0953, grad_fn=<DivBackward1>)
Enter
tensor(1.0959, grad_fn=<DivBackward1>)
Enter
tensor(1.0939, grad_fn=<DivBackward1>)
Enter
tensor(1.0959, grad_fn=<DivBackward1>)
Enter
tensor(1.0962, grad_fn=<DivBackward1>)
Enter
tensor(1.0936, grad_fn=<DivBackward1>)
Enter
tensor(1.0960, grad_fn=<DivBackward1>)
Enter
tensor(1.0976, grad_fn=<DivBackward1>)
Enter
tensor(1.0967, grad_fn=<DivBackward1>)
Enter
tensor(1.0941, grad_fn=<DivBackward1>)
Enter
tens

In [ ]:
p = torch.tensor([0.33,0.33,0.34]).reshape(-1,3)
out = model_pred(p)
print(out)

(tensor([[0.2046, 0.4349, 0.3605]], grad_fn=<SoftmaxBackward0>), tensor([-0.1303], grad_fn=<SelectBackward0>))


<ipython-input-8-9e42e92374fe>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(out[:,0:3])


In [ ]:
import matplotlib.pyplot as plt
p_list = []
rewards = []
for i in range(200):
    p = generate_p()
    p =np.random.permutation(p)


    p_1 = torch.tensor(p,dtype=torch.float).reshape(-1,3)
    out,_ = model_pred(p_1)
    r = compute_reward(out.reshape(-1),p)
    rewards.append(r)
    p_list.append(p)
#plt.plot(p_list,rewards)



<ipython-input-8-9e42e92374fe>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(out[:,0:3])


KeyboardInterrupt: ignored

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import mpl_toolkits.mplot3d
import matplotlib


fig = plt.figure()
#ax = fig.gca(projection='3d')
ax = fig.add_subplot(111, projection='3d')
#fig.gca(projection='3d')
p_arr = np.array(p_list)
x = p_arr[:,0]
y = p_arr[:,1]
z = p_arr[:,2]
re = np.array(rewards)
print(re.shape)

img = ax.scatter(x,y,z,c=re, cmap=plt.hot())
fig.colorbar(img,location="left")
ax.set_xlabel('rock')
ax.set_ylabel('paper')
ax.set_zlabel('scissors')
ax.zaxis.labelpad=-0.7
plt.show()